In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

import numpy as np

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from registration_utilities import *
from annotation_utilities import *
from metadata import *
from data_manager import *
from visualization_utilities import *

Setting environment for Local Macbook Pro


# Draw globally and locally warped atlas on the same section.

In [2]:
stack_fixed = 'MD585'
stack_moving = 'atlasV5'
detector_id = 15
downsample_factor = 8
bg_img_version = 'grayJpeg'

In [3]:
volume_downsample = 32

warped_volumes_global = DataManager.load_transformed_volume_all_known_structures(stack_m=stack_moving, 
                                                                          stack_f=stack_fixed, 
                                                                          prep_id_f=2,
                                                                        detector_id_f=detector_id,
                                                                        warp_setting=20,
                                                                          downscale=volume_downsample,
                                                                          trial_idx=None, sided=True)

if len(warped_volumes_global) == 0:
    raise Exception('No volumes are loaded.')

    
warped_volumes_local = DataManager.load_transformed_volume_all_known_structures(stack_m=stack_moving, 
                                                                          stack_f=stack_fixed, 
                                                                          prep_id_f=2,
                                                                        detector_id_f=detector_id,
                                                                        warp_setting=17,
                                                                          downscale=volume_downsample,
                                                                          trial_idx=None, sided=True)

if len(warped_volumes_local) == 0:
    raise Exception('No volumes are loaded.')

# For getting correct contour location
bbox_down32 = \
DataManager.load_original_volume_bbox(stack=stack_fixed, volume_type='score', structure='7N', 
                                      downscale=32, prep_id=2, detector_id=detector_id)
xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f = bbox_down32 * 32 / volume_downsample
print xmin_vol_f, xmax_vol_f, ymin_vol_f, ymax_vol_f, zmin_vol_f, zmax_vol_f

Prior structure/index map not found. Generating a new one.
Prior structure/index map not found. Generating a new one.


0 511 0 374 111 479


In [4]:
ydim_f, xdim_f, zdim_f = warped_volumes_global['10N_L'].shape
centroid_f = np.array([xdim_f/2, ydim_f/2, zdim_f/2])

In [19]:
warped_volumes_original = DataManager.load_original_volume_all_known_structures(stack=stack_moving, 
                                                                          downscale=volume_downsample,
                                                                          sided=True,
                                                                                return_structure_index_mapping=False
                                                                           )

Volume shape: (356, 427, 375)


In [20]:
ydim_m, xdim_m, zdim_m = warped_volumes_original['10N_L'].shape
centroid_m = np.array([xdim_m/2, ydim_m/2, zdim_m/2])

In [21]:
T = np.column_stack([np.eye(3), -centroid_m + centroid_f])

warped_volumes_centroidAlignedToFixed_tuples = {}

for sided_name, vol in warped_volumes_original.iteritems():
    warped_volumes_centroidAlignedToFixed_tuples[sided_name] = transform_volume_v2(vol, T.flatten())

Interpolating/filling sparse volume: 0.02 seconds.
Interpolating/filling sparse volume: 0.08 seconds.
Interpolating/filling sparse volume: 0.04 seconds.
Interpolating/filling sparse volume: 0.02 seconds.
Interpolating/filling sparse volume: 0.01 seconds.
Interpolating/filling sparse volume: 0.04 seconds.
Interpolating/filling sparse volume: 0.00 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.04 seconds.
Interpolating/filling sparse volume: 0.01 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.11 seconds.
Interpolating/filling sparse volume: 0.05 seconds.
Interpolating/filling sparse volume: 0.03 seconds.
Interpolating/filling sparse volume: 0.05 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.03 seconds.
Interpolating/filling sparse volume: 0.09 seconds.
Interpolating/filling sparse volume: 0.05 seconds.
Interpolating/filling sparse vo

In [22]:
del warped_volumes_original

In [23]:
warped_volumes_centroidAlignedToFixed = crop_and_pad_volumes(vol_bbox_dict=warped_volumes_centroidAlignedToFixed_tuples, 
                     out_bbox=(0,xdim_f-1, 0,ydim_f-1, 0,zdim_f-1))

In [24]:
del warped_volumes_centroidAlignedToFixed_tuples

In [1]:
scoremap_viz = generate_scoremap_layer(stack=stack_fixed, structure='5N', downscale=8, detector_id=15, sec=155)

NameError: name 'generate_scoremap_layer' is not defined

In [26]:
sec = 155

viz = annotation_from_multiple_warped_atlases_overlay_on(bg=scoremap_viz, 
                                                                warped_volumes_sets={
                                                                    'centerAligned': warped_volumes_centroidAlignedToFixed,
                                                                    'global': warped_volumes_global,
                                                                                'local': warped_volumes_local},
                                              volume_origin=(xmin_vol_f, ymin_vol_f, zmin_vol_f),
                                              stack_fixed=stack_fixed, sec=sec, out_downsample=downsample_factor,
                                             bg_img_version=bg_img_version,
                                             levels=[0.5],
                                                        level_colors={'centerAligned': {0.5: (0,0,255)},
                                                                    'global': {0.5: (255,0,0)},
                                                                     'local': {0.5: (0,255,0)}})

/home/yuncong/Brain/utilities/visualization_utilities.py:211: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if bg == 'original':


local Sp5C_L
local VLL_L
local 7N_L
local LRt_L
local SNR_L
local Sp5O_L
local 5N_L
local SNC_L
local IC
local 7n_L
local SC
local Amb_L
global Sp5C_L
global VLL_L
global 7N_L
global LRt_L
global SNR_L
global 5N_L
global SNC_L
global IC
global DC_L
global 7n_L
global SC
global Amb_L
centerAligned Sp5C_L
centerAligned VLL_L
centerAligned 7N_L
centerAligned LRt_L
centerAligned SNR_L
centerAligned 5N_L
centerAligned SNC_L
centerAligned IC
centerAligned DC_L
centerAligned 7n_L
centerAligned SC
centerAligned Amb_L


In [63]:
sec = 155

viz = annotation_from_multiple_warped_atlases_overlay_on(bg='original', 
                                                                warped_volumes_sets={
                                                                    'centerAligned': warped_volumes_centroidAlignedToFixed,
                                                                    'global': warped_volumes_global,
                                                                                'local': warped_volumes_local},
                                              volume_origin=(xmin_vol_f, ymin_vol_f, zmin_vol_f),
                                              stack_fixed=stack_fixed, sec=sec, out_downsample=downsample_factor,
                                             bg_img_version=bg_img_version,
                                             levels=[0.5],
                                                        level_colors={'centerAligned': {0.5: (0,0,255)},
                                                                    'global': {0.5: (255,0,0)},
                                                                     'local': {0.5: (0,255,0)}})

LRt_R
Sp5C_L
Pn_L
7N_R
PBG_L
VLL_L
4N_R
VLL_R
7N_L
PBG_R
RtTg
Sp5C_R
Pn_R
LRt_L
Sp5O_R
SNR_L
12N
SNR_R
Sp5O_L
DC_R
VCA_L
3N_L
6N_R
Sp5I_L
RMC_R
5N_L
SNC_L
SNC_R
AP
RMC_L
Sp5I_R
3N_R
6N_L
IC
VCA_R
DC_L
Amb_R
VCP_R
LC_R
10N_R
7n_R
5N_R
Tz_L
Tz_R
7n_L
4N_L
LC_L
10N_L
SC
VCP_L
Amb_L
LRt_R
Sp5C_L
Pn_L
7N_R
PBG_L
VLL_L
4N_R
VLL_R
7N_L
PBG_R
RtTg
Sp5C_R
Pn_R
LRt_L
Sp5O_R
SNR_L
12N
SNR_R
Sp5O_L
DC_R
VCA_L
3N_L
6N_R
Sp5I_L
RMC_R
5N_L
SNC_L
SNC_R
AP
RMC_L
Sp5I_R
3N_R
6N_L
IC
VCA_R
DC_L
Amb_R
VCP_R
LC_R
10N_R
7n_R
5N_R
Tz_L
Tz_R
7n_L
4N_L
LC_L
10N_L
SC
VCP_L
Amb_L
LRt_R
Sp5C_L
Pn_L
7N_R
PBG_L
VLL_L
4N_R
VLL_R
7N_L
PBG_R
RtTg
Sp5C_R
Pn_R
LRt_L
Sp5O_R
SNR_L
12N
SNR_R
Sp5O_L
DC_R
VCA_L
3N_L
6N_R
Sp5I_L
RMC_R
5N_L
SNC_L
SNC_R
AP
5N_R
Sp5I_R
3N_R
6N_L
IC
VCA_R
DC_L
Amb_R
VCP_R
LC_R
10N_R
7n_R
RMC_L
Tz_L
Tz_R
7n_L
4N_L
LC_L
10N_L
SC
VCP_L
Amb_L


In [27]:
display_image(viz)

/home/yuncong/Brain/registration/tmp.jpg

In [66]:
sec = 165

viz = annotation_from_multiple_warped_atlases_overlay_on(bg='original', 
                                                                warped_volumes_sets={
                                                                    'centerAligned': warped_volumes_centroidAlignedToFixed,
                                                                    'global': warped_volumes_global,
                                                                                'local': warped_volumes_local},
                                              volume_origin=(xmin_vol_f, ymin_vol_f, zmin_vol_f),
                                              stack_fixed=stack_fixed, sec=sec, out_downsample=downsample_factor,
                                             bg_img_version=bg_img_version,
                                             levels=[0.5],
                                                        level_colors={'centerAligned': {0.5: (0,0,255)},
                                                                    'global': {0.5: (255,0,0)},
                                                                     'local': {0.5: (0,255,0)}})

local Sp5C_L
local Pn_L
local VLL_L
local 7N_L
local LRt_L
local SNR_L
local SNC_L
local IC
local 7n_L
local SC
global Sp5C_L
global VLL_L
global 7N_L
global LRt_L
global SNR_L
global 5N_L
global SNC_L
global IC
global 7n_L
global SC
centerAligned Sp5C_L
centerAligned Pn_L
centerAligned VLL_L
centerAligned 7N_L
centerAligned LRt_L
centerAligned SNR_L
centerAligned SNC_L
centerAligned IC
centerAligned 7n_L
centerAligned SC


In [67]:
display_image(viz)

/home/yuncong/Brain/registration/tmp.jpg